# Exemplo 17: Ride-Sharing - Preço Dinâmico (Spark)

Este notebook demonstra o cálculo de preço dinâmico (Surge Pricing) em tempo real para aplicativo de transporte usando **Spark Streaming**.

**Cenário**: Calcular multiplicador de preço por região baseado na demanda (número de solicitações) nos últimos minutos.

## 1. Configuração

In [ ]:
# Instalar Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixar e Instalar Spark
!wget https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz && tar xf spark-3.5.0-bin-hadoop3.tgz

# Baixar e Instalar Kafka
!wget https://archive.apache.org/dist/kafka/3.6.1/kafka_2.13-3.6.1.tgz && tar xf kafka_2.13-3.6.1.tgz

# Instalar pacotes Python
!pip install -q findspark pyspark kafka-python

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
import findspark
findspark.init()

## 2. Iniciar Kafka

In [ ]:
%%bash
cd kafka_2.13-3.6.1
bin/zookeeper-server-start.sh -daemon config/zookeeper.properties
sleep 5
bin/kafka-server-start.sh -daemon config/server.properties
sleep 5
bin/kafka-topics.sh --create --topic ride-requests --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1

## 3. Simulador de Solicitações

In [ ]:
import json
import time
import random
from kafka import KafkaProducer
import threading
from datetime import datetime

def generate_ride_requests():
    producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                             value_serializer=lambda x: json.dumps(x).encode('utf-8'))
    regions = ['Downtown', 'Airport', 'Suburb', 'Beach']
    
    try:
        for _ in range(500):
            # Alta demanda no Aeroporto
            region = np.random.choice(regions, p=[0.2, 0.5, 0.2, 0.1]) 
            data = {'region': region, 'user_id': random.randint(1, 1000), 'timestamp': datetime.now().isoformat()}
            producer.send('ride-requests', value=data)
            time.sleep(0.05)
    finally:
        producer.close()

import numpy as np
thread = threading.Thread(target=generate_ride_requests)
thread.start()

## 4. Cálculo de Preço Dinâmico

In [ ]:
%%writefile kafka_consumer.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, window, count, when
from pyspark.sql.types import StructType, StructField, StringType, TimestampType

spark = SparkSession.builder.appName("DynamicPricing").getOrCreate()

schema = StructType([
    StructField("region", StringType()),
    StructField("user_id", StringType()),
    StructField("timestamp", TimestampType())
])

df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "ride-requests") \
    .load()

requests = df.select(from_json(col("value").cast("string"), schema).alias("data")).select("data.*")

# Contar solicitações por região em janelas de 10 segundos
demand = requests \
    .withWatermark("timestamp", "10 seconds") \
    .groupBy(
        window(col("timestamp"), "10 seconds"),
        col("region")
    ) \
    .count()

# Definir Multiplicador
# Se count > 50 -> 2.0x, > 20 -> 1.5x, else 1.0x
pricing = demand.withColumn("multiplier", 
    when(col("count") > 50, 2.0)
    .when(col("count") > 20, 1.5)
    .otherwise(1.0)
)

query = pricing.writeStream \
    .outputMode("update") \
    .format("console") \
    .option("truncate", "false") \
    .start()


query.awaitTermination()

In [ ]:
!spark-submit --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 kafka_consumer.py